In [8]:
import sys
sys.path.append('../')
import pandas as pd 
from lib.path import data_file_path, ensure_data_dir
from lib.columns import clean_column_names
from lib.clean import clean_dates


def clean_tracking_number(df):
    df.tracking_number = df.tracking_number.fillna('').str.lower().str.strip()\
        .str.replace(r'(ia-|ia )', '', regex=True)\
        .str.replace('.', '-', regex=False)\
        .str.replace('2018', '18', regex=False)\
        .str.replace('1801', '18-01', regex=False)
    return df


def clean():
    df = pd.read_csv(data_file_path('hammond_pd/hammond_pd_cprr_2015_2020.csv'))\
        .pipe(clean_column_names)
    df = df\
        .rename(columns={
            'investigation_date_start': 'investigation_start_date',
        })\
        .pipe(clean_tracking_number)\
        .pipe(clean_dates, ['investigation_start_date', 'incident_date'])
    return df


if __name__ == '__main__':
    df = clean()
    ensure_data_dir('clean')
    df.to_csv(data_file_path('clean/cprr_hammond_pd_2015_2020.csv'), index=False)


ValueError: unknown date format "7/2010 til 5/2012"

In [ ]:
df.incident_date.unique()

array(['17-Oct-15', '25-May-15', '29-Jan-15', '16-Oct-16', '28-Oct-16',
       '28-Aug-17', '11-Dec-17', '20-May-17', '9-Apr-18',
       '7/2010 til 5/2012', '10-Jul-18', '31-Dec-18', '31-Aug-18',
       '23-Apr-19', '28-May-19', '7-Nov-19', '8-Sep-20', '20-Mar-20'],
      dtype=object)